## Homework 1

### Question 1

#### Q1.1

Import required packages.

In [84]:
# import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install sklearn
import numpy as np
import pickle
from sklearn import preprocessing

Create `myknn_regressor` class.

In [85]:
class myknn_regressor():
    
    def __init__(self, n_neighbors = 10, mean_type = "equal_weight"):
        """
        mean_type can be equal_weight or remove_outliers.
        equal_weight use the same weight for all neighbors.
        remove_outliers remove neighbors out in [Q1 - 1.5 IQR, Q3 + 1.5IQR].
        """
        self.n_neighbors = n_neighbors
        self.mean_type = mean_type

    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def predict(self, x_test):
        """use remove_outliers only if k>=10"""
        
        y_pred = []
        count = 0
        for x in x_test:
            dist = np.linalg.norm(x - self.x_train, axis=1)
            selected_idx = np.argsort(dist, kind='stable')[:self.n_neighbors]
            
            if self.mean_type == 'remove_outliers' and self.n_neighbors >= 10:
                # remove outliers
                q1 = np.quantile(self.y_train[selected_idx], .25)
                q3 = np.quantile(self.y_train[selected_idx], .75)
                iqr = q3 - q1
                def notOutlier(idx):
                    return ((self.y_train[idx] >= q1 - 1.5*iqr) and (self.y_train[idx] <= q3 + 1.5*iqr))
                selected_idx = list(filter(notOutlier, selected_idx))
                 
            pred = np.mean(self.y_train[selected_idx])
            y_pred.append(pred)
        
        return np.array(y_pred)
                

#### Q1.2
Load and standardize data.

In [86]:
with open('msd_data1.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
x_train = xscaler.transform(msd_data['X_train'])
x_test = xscaler.transform(msd_data['X_test'])
y_train = msd_data['Y_train']
y_test = msd_data['Y_test']

Fit the KNN regressor and predict.

In [87]:
myknn = myknn_regressor(20, 'equal_weight')
myknn.fit(x_train, y_train)
y_pred = myknn.predict(x_test)

Compute RMSE.

In [88]:
def rmse(ground_truth, prediction):
    mse = np.mean((ground_truth - prediction) ** 2)
    return np.sqrt(mse)
print(f'RMSE = {rmse(y_test, y_pred)}')

RMSE = 10.25126451549596


List the first 20 predictions.

In [89]:
np.set_printoptions(precision=2)
print(f'Predictions: {y_pred[:20]}')

Predictions: [1993.35 1993.8  2000.65 1991.5  1992.8  1998.5  1988.1  1991.65 2002.25
 2003.   2000.5  1998.65 1995.55 1997.2  1995.05 1997.4  1992.15 2000.45
 2003.2  1995.75]


#### Q1.3

Data have been load and standardized in the previous section.

Fit the KNN model with type 'remove outlier' and predict.

In [90]:
myknn = myknn_regressor(20, 'remove_outliers')
myknn.fit(x_train, y_train)
y_pred = myknn.predict(x_test)

Compute RMSE.

In [91]:
print(f'RMSE = {rmse(y_test, y_pred)}')

RMSE = 10.212572466080376


List the first 20 predictions.

In [92]:
print(f'Predictions: {y_pred[:20]}')

Predictions: [1993.35 1993.8  2000.65 1992.74 1992.8  2000.   1988.1  1991.65 2002.25
 2003.95 2000.5  2000.94 1995.55 1997.2  1998.61 1997.4  1992.15 2003.83
 2003.2  1995.75]
